
# Finding answers

In [ ]:
import pandas as pd
import seaborn as sns

from common_code import data_folder

In [ ]:
sales_df = pd.read_csv(f'{data_folder}/sales2020.csv')

## Channel profits

In [ ]:
channel_profits_df = (
    sales_df
    .groupby('Channel')
    .agg({
        'Quantity': 'sum',
        'Gross Profit': 'sum', 
        'Margin per Unit': 'mean'
    })
    .round(3)
    .sort_values('Gross Profit', ascending=False)
)

In [ ]:
channel_profits_df

In [ ]:
channel_profits_df.plot(
    kind='bar', figsize=(15, 4), subplots=True,
    layout=(1, 3), legend=False, rot=30
);

## Category profits

In [ ]:
category_profits_df = (
    sales_df
    .groupby('Category')
    .agg({
        'Quantity': 'sum',
        'Gross Profit': 'sum', 
        'Margin per Unit': 'mean'
    })
    .round(3)
    .sort_values('Gross Profit', ascending=False)
)

In [ ]:
category_profits_df

In [ ]:
category_profits_df.plot( 
    kind='barh', figsize=(15, 6), subplots=True,
    layout=(1, 3), legend=False, sharex=False, sharey=True
)

## Channel and category profits

In [ ]:
average_margin_per_category = pd.pivot_table(
    sales_df, 
    index='Channel', 
    columns='Category', 
    values='Margin per Unit',
    aggfunc='mean'
).round(3)

In [ ]:
average_margin_per_category

In [ ]:
sns.heatmap(
    data=average_margin_per_category.transpose(), 
    cmap='Reds', 
    annot=True, 
    fmt=".2f", 
    linewidths=.5,
    cbar=False
)

## Product profits

In [ ]:
def get_product_profits(df, category):
    return df[df['Category'] == category]

In [ ]:
get_product_profits(sales_df, 'Toys & Games')

In [ ]:
def get_product_profits(df, category):
    
    df = df[df['Category'] == category]

    return (
        df.groupby('ProductID')
          .agg({
                'Product Name': 'first',
                'Unit Price': 'first',
                'Category': 'first',
                'Quantity': 'sum',
                'Gross Profit': 'sum',
                'Margin per Unit': 'mean'
            })
          .sort_values(by='Gross Profit', ascending=False)
          .reset_index()
          .round(3)
    )

In [ ]:
get_product_profits(sales_df, 'Toys & Games')

In [ ]:
def get_product_profits(df, 
                        category='All', channel='All', 
                        sort_column='Gross Profit', ascending=False):

    if category != 'All':
        df = df[df['Category'] == category]

    if channel != 'All':
        df = df[df['Channel'] == channel]

    return (
        df.groupby('ProductID')
          .agg({
                'Product Name': 'first',
                'Unit Price': 'first',
                'Category': 'first',
                'Quantity': 'sum',
                'Gross Profit': 'sum',
                'Margin per Unit': 'mean'
            })
          .sort_values(by=sort_column, ascending=ascending)
          .reset_index()
          .round(3)
    )

In [ ]:
get_product_profits(sales_df, 
                    category='Toys & Games', 
                    channel='iBay.com', 
                    sort_column='Margin per Unit', 
                    ascending=True)

In [ ]:
with pd.ExcelWriter(f'{data_folder}/Product profits 2020.xlsx') as writer:
    for category in sales_df['Category'].unique():
        products_df = get_product_profits(sales_df, category)
        products_df.to_excel(writer, sheet_name=category, index=False)

## Sharing results

### Exporting notebooks

In [ ]:
!jupyter-nbconvert "03 - Finding answers.ipynb" --no-input --output "Sales analysis.html";

### Overthinking: Creating an interactive dashboard

In [ ]:
import pandas as pd

import hvplot.pandas
import panel as pn
pn.extension();

from common_code import data_folder, get_product_profits

In [ ]:
sales_df = pd.read_csv(f'{data_folder}/sales2020.csv')

In [ ]:
def products_scatterplot(category='All', channel='All'):    
    df = get_product_profits(sales_df, category, channel)
    
    return df.hvplot(
        kind='scatter',
        x='Gross Profit', 
        y='Margin per Unit', 
        size='Quantity', 
        color='Category',
        scale=0.2, 
        grid=True,
        line_color='black', 
        width=900, 
        height=600, 
        hover_cols=['ProductID', 'Product Name', 'Unit Price']
    )

In [ ]:
products_scatterplot()

In [ ]:
test_dropdown = pn.widgets.Select(name='Test dropdown', value='a', options=['a', 'b', 'c'])

In [ ]:
test_dropdown

In [ ]:
channels = ['All'] + sorted(sales_df['Channel'].unique())
channel_dropdown = pn.widgets.Select(name='Channel', value='All', options=channels)

In [ ]:
channel_dropdown

In [ ]:
categories = ['All'] + sorted(sales_df['Category'].unique())
category_dropdown = pn.widgets.Select(name='Category', value='All', options=categories)

In [ ]:
category_dropdown

In [ ]:
@pn.depends(category_dropdown, channel_dropdown)
def products_scatterplot(category='All', channel='All'):    
    df = get_product_profits(sales_df, category, channel)
    
    return df.hvplot(
        kind='scatter',
        x='Gross Profit', 
        y='Margin per Unit', 
        size='Quantity', 
        color='Category',
        scale=0.2, 
        grid=True,
        line_color='black', 
        width=900, 
        height=600, 
        hover_cols=['ProductID', 'Product Name', 'Unit Price']
    )

In [ ]:
pn.Column(
    category_dropdown,
    channel_dropdown,
    products_scatterplot
)

In [ ]:
dashboard = pn.Column(
    "# Products dashboard",
    "Select sales channel or product category using the menus below:",
    category_dropdown,
    channel_dropdown,
    products_scatterplot
)

In [ ]:
dashboard.show()

In [ ]:
dashboard.save('Products dashboard.html', embed=True)